**LinkedIn Profile:** [Matencio Montana](https://www.linkedin.com/in/montana-matencio-b01111376)  
**Contact:** [montana.matencio@gmail.com](mailto:montana.matencio@gmail.com)


**Rock Paper Scissors Classification: A Deep Learning Approach**


**Project Overview**
This project explores the exciting use of deep learning in computer vision. It involves building and testing models that can classify hand gestures for the game of Rock, Paper, and Scissors. By training on a special dataset of hand images, the goal is to create smart systems that accurately recognize these gestures.

**Technologies & Methodologies**
This project leverages a comprehensive suite of Python libraries and deep learning methodologies:

- Python: The core programming language underpinning the entire project.

- Pandas & NumPy: Utilized for efficient data loading, manipulation, and numerical operations, particularly in handling the dataset's metadata (CSV files).

- PyTorch: The chosen deep learning framework, providing the flexibility and power to define, train, and evaluate neural network architectures.

- CNN (Convolutional Neural Network): The fundamental deep learning architecture implemented (RockPaperScissorsCNN), specifically designed for its superior performance in image recognition and classification tasks.

- Residual Networks (ResNet): A key architectural enhancement, implemented through ResBlock components and integrated into RockPaperScissorsResNetwork, allowing for the training of deeper networks by mitigating the vanishing gradient problem and improving learning efficiency.

- torchvision.transforms: Used for crucial image preprocessing steps, including resizing images to a uniform IMAGE_SIZE (set to 16x16 pixels), and applying data augmentation techniques like RandomHorizontalFlip, RandomRotation(10), and ColorJitter to enhance model generalization and robustness.

- Batch Normalization & Dropout: Integrated within both CNN and ResNet architectures (nn.BatchNorm2d, nn.Dropout) to improve training stability, accelerate convergence, and serve as effective regularization techniques to prevent overfitting.

- DataLoader and Dataset: Custom RockPaperScissorsDataset and DataLoader classes are implemented for efficient batching and loading of image data during training and evaluation.

- Loss Function & Optimizer: nn.CrossEntropyLoss is used as the loss function for multi-class classification, and torch.optim.AdamW is employed as the optimizer for model training.

- Learning Rate Scheduler: lr_scheduler.CosineAnnealingLR is used to dynamically adjust the learning rate during training, further optimizing model convergence.

**Data Source**
The models are trained and evaluated on a specialized, Open Source dataset containing images of hands forming Rock, Paper, and Scissors gestures. This dataset is organized with separate training and test directories (rock-paper-scissors/train/train/, rock-paper-scissors/test/test/), accompanied by annotation CSV files (_annotations.csv), which facilitate structured data loading and labeling. The images in this dataset were collected from various sources and augmented to create a diverse dataset. The dataset used is "Rock-Paper-Scissors" by Adil Shamim, available on Kaggle: https://www.kaggle.com/datasets/adilshamim8/rock-paper-scissors.

**Notebook/Project Structure**

- Data Loading & Preprocessing: This initial phase involves reading annotation CSV files using Pandas, inspecting data distribution, and defining torchvision.transforms for image resizing, normalization, and data augmentation. A custom RockPaperScissorsDataset class handles image loading and label mapping.

- Dataset & DataLoader Creation: PyTorch DataLoader objects are created for both training and evaluation sets, enabling efficient batch processing of image data.

- Model Definition: Two distinct Convolutional Neural Network architectures are defined and explored:

    - RockPaperScissorsCNN: A simpler, sequential CNN model.

    - RockPaperScissorsResNetwork: A more advanced network incorporating ResBlock for residual connections, enabling deeper and more powerful feature extraction.

- Training & Evaluation Loops: Dedicated training_loop and evaluate_loop functions manage the training iterations, forward passes, backpropagation, optimizer steps, and performance monitoring (loss and accuracy).

- Model Training & Hyperparameters: Both models are trained for 50 epochs using CrossEntropyLoss and AdamW optimizer, with learning rate scheduling.

**Key Findings & Model Performance**
After 50 epochs of training, significant differences in performance were observed between the two architectures, demonstrating that both models are effectively learning to classify hand gestures, performing significantly better than random chance (which would yield approximately 33.3% accuracy for three classes):

**RockPaperScissorsCNN Model:**

- Test Accuracy: 52.5%

- Average Test Loss: 1.040739

- This model achieved a moderate accuracy, indicating its ability to classify gestures, but with clear room for improvement.

**RockPaperScissorsResNetwork Model:**

- Test Accuracy: 73.5%

- Average Test Loss: 1.006295

- The Residual Network significantly outperformed the simpler CNN, demonstrating a much higher accuracy. This highlights the effectiveness of residual connections in learning more complex features from the image data, even with small image sizes.

These results underscore the importance of architectural choices in deep learning, with the ResNet-inspired design proving more effective for this image classification task.

**Next Steps & Future Work**
Building upon the insights gained from training these custom architectures, the logical next step to further enhance model performance and generalization is to implement transfer learning. This will involve:

- Utilizing a pre-trained ResNet model: Leveraging a state-of-the-art ResNet architecture (e.g., ResNet18, ResNet50) pre-trained on a large-scale dataset like ImageNet.

- Fine-tuning the model: Adapting the pre-trained model to the specific Rock Paper Scissors classification task by replacing the final classification layer and retraining it on the dataset, potentially with a lower learning rate for earlier layers. This approach is expected to significantly boost accuracy by leveraging features learned from a vast array of images.

**Reproducibility**
- Random seeds are set at the beginning of the script to ensure the reproducibility of results.

In [13]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.optim import lr_scheduler

In [106]:
import random
import numpy as np
import os # To set environment variables, useful for some libraries

def set_seed(seed):
    """
    Sets the random seed for reproducibility across different libraries.
    """
    # 1. Set seed for Python's built-in random module
    random.seed(seed)

    # 2. Set seed for NumPy
    np.random.seed(seed)

    # 3. Set seed for PyTorch (CPU and GPU)
    torch.manual_seed(seed) # For CPU operations
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed) # For current GPU
        torch.cuda.manual_seed_all(seed) # For all GPUs (if you have multiple)

    # 4. Ensure deterministic behavior for CuDNN (GPU operations)
    #    This can sometimes slightly slow down training, but ensures exact reproducibility.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # Disable CuDNN auto-tuner for deterministic ops

    # 5. Set environment variable for Python hashing (affects dicts, sets, etc.)
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to {seed} for all relevant libraries.")

MY_RANDOM_SEED = 42
set_seed(MY_RANDOM_SEED)

Random seed set to 42 for all relevant libraries.


In [15]:
train_csv_file_path = 'rock-paper-scissors/train/train/_annotations.csv'
train_raw_df = pd.read_csv(train_csv_file_path)

print(train_raw_df.head())
print(train_raw_df.filename[0])
print(train_raw_df.loc[:,'class'].isnull().sum())#So all the images are labelled

print(train_raw_df.groupby('class').size()) #So all images correctly belong to either Papper, Rock, Scissors 

                                            filename  width  height  class  \
0  egohands-public-1620914960773_png_jpg.rf.aa184...    640     640   Rock   
1  egohands-public-1624053434391_png_jpg.rf.aaef5...    640     640  Paper   
2  egohands-public-1624465902684_png_jpg.rf.aaa09...    640     640   Rock   
3  Screen-Shot-2022-02-08-at-12-59-24-PM_png.rf.a...    640     640   Rock   
4  egohands-public-1622127402076_png_jpg.rf.aa897...    640     640   Rock   

   xmin  ymin  xmax  ymax  
0   429   185   562   319  
1   269   354   544   443  
2   427   332   551   509  
3    80   268   145   395  
4    83   128   296   381  
egohands-public-1620914960773_png_jpg.rf.aa184eeebad98b2fb04354d01a90b9d0.jpg
0
class
Paper       1349
Rock        1924
Scissors    1337
dtype: int64


In [62]:
train_df = train_raw_df.loc[:, ['filename','class']]
print(train_df.info())
print(train_df.loc[0,['filename']])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4610 entries, 0 to 4609
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  4610 non-null   object
 1   class     4610 non-null   object
dtypes: object(2)
memory usage: 72.2+ KB
None
filename    egohands-public-1620914960773_png_jpg.rf.aa184...
Name: 0, dtype: object


In [17]:
evaluate_csv_file_path ='rock-paper-scissors/test/test/_annotations.csv'

evaluate_raw_df = pd.read_csv(evaluate_csv_file_path)

print(evaluate_raw_df.info())
print(evaluate_raw_df.loc[:,'class'].isnull().sum())#So all the images are labelled

print(evaluate_raw_df.groupby('class').size()) #So all images correctly belong to either Papper, Rock, Scissors


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  204 non-null    object
 1   width     204 non-null    int64 
 2   height    204 non-null    int64 
 3   class     204 non-null    object
 4   xmin      204 non-null    int64 
 5   ymin      204 non-null    int64 
 6   xmax      204 non-null    int64 
 7   ymax      204 non-null    int64 
dtypes: int64(6), object(2)
memory usage: 12.9+ KB
None
0
class
Paper       72
Rock        65
Scissors    67
dtype: int64


In [18]:
evaluate_df = evaluate_raw_df.loc[:, ['filename','class']]
print(evaluate_df.info())
print(evaluate_df.loc[0,['filename']])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  204 non-null    object
 1   class     204 non-null    object
dtypes: object(2)
memory usage: 3.3+ KB
None
filename    IMG_7079_MOV-23_jpg.rf.123a8de8c8da646e4a25f1c...
Name: 0, dtype: object


In [19]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using {device} device")

Using cuda device


In [20]:
import torchvision
import torchvision.transforms as transforms
# --- Image dimensions for thr model ---
IMAGE_SIZE = 16


# --- 1. Define Transformations (same as before, but applied within custom dataset) ---
train_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), #for our model to work, it's mandatory to have same size images
    transforms.RandomHorizontalFlip(),#a way to artificially increase the dataset, and improve generaliztion
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),

])


In [21]:
from pathlib import Path
from PIL import Image
#Datasets

class RockPaperScissorsDataset(Dataset):

    def __init__(self,df, img_file_path, transform):
        self.df = df
        self.img_file_path = img_file_path
        self.transform = transform
        self.class_to_value = {'Rock': 0, 'Paper': 1, 'Scissors': 2}
        self.list_images_tensors =[]
        for idx in range(len(df)):
            # Open the image
            image = Image.open(self.img_file_path + self.df.loc[idx,'filename'])
            # Ensure it's RGB (important for consistency)
            image = image.convert('RGB')
            image_tensor = self.transform(image)
            image.close()
            self.list_images_tensors.append(image_tensor)
            
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        image_tensor = self.list_images_tensors[idx]
        ground_truth= self.class_to_value[self.df.loc[idx,'class']]

        return image_tensor,ground_truth
train_dataset= RockPaperScissorsDataset(train_df,'rock-paper-scissors/train/train/',train_transforms)

evaluate_dataset = RockPaperScissorsDataset(evaluate_df, 'rock-paper-scissors/test/test/', test_transforms)


Printing the result to make sure: that the images are correctly transformed into tensors, and 'class' into a integer from 0 to 2

In [11]:
print(train_dataset[2][0], train_dataset[2][1])


tensor([[[0.8706, 0.8824, 0.8902, 0.8627, 0.8627, 0.9059, 0.6549, 0.4510,
          0.4863, 0.4471, 0.6980, 0.7098, 0.6706, 0.6471, 0.6275, 0.6275],
         [0.8980, 0.9294, 0.9412, 0.9059, 0.8784, 0.8510, 0.5686, 0.5843,
          0.5765, 0.4863, 0.5529, 0.6549, 0.6784, 0.7059, 0.6980, 0.7137],
         [0.8667, 0.8824, 0.9529, 0.9569, 0.8745, 0.6824, 0.5490, 0.5529,
          0.5333, 0.5176, 0.5608, 0.7608, 0.7765, 0.7882, 0.7843, 0.7725],
         [0.8353, 0.7451, 0.8471, 0.9098, 0.8627, 0.6824, 0.5961, 0.5804,
          0.5647, 0.5804, 0.5686, 0.7490, 0.7373, 0.7725, 0.8078, 0.8314],
         [0.8902, 0.8118, 0.8314, 0.8039, 0.6863, 0.6353, 0.6314, 0.6471,
          0.6157, 0.6745, 0.6431, 0.9373, 0.9843, 1.0000, 1.0000, 1.0000],
         [0.8392, 0.7882, 0.8431, 0.8745, 0.7373, 0.6510, 0.5882, 0.6588,
          0.6196, 0.6745, 0.6902, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [1.0000, 0.9333, 1.0000, 1.0000, 0.8706, 0.7490, 0.5137, 0.6784,
          0.6549, 0.6314, 0.6941

In [22]:
batch_size = 64
#let's make the dataloaders 
training_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
evaluate_dataloader = DataLoader(evaluate_dataset, batch_size=batch_size, shuffle=False)

for X,y in training_dataloader:
    print(X.shape)
    print(y.shape)
    break 

torch.Size([64, 3, 16, 16])
torch.Size([64])


In [23]:
class RockPaperScissorsCNN(nn.Module):
    def __init__(self, input_channels, hidden_channels, output_size, images_size, dropout_prob):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(input_channels,hidden_channels, kernel_size=(3,3), stride=(1,1), padding=(1,1) ),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(dropout_prob),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(hidden_channels * images_size[0]//2 * images_size[1]//2, output_size)
        )

    def forward(self, x):
        logits = self.cnn(x)
        return logits 



In [22]:
class ResBlock(nn.Module):
    def __init__(self, input_channels, hidden_channels):
        super().__init__(),
        self.convs = nn.Sequential(
            nn.Conv2d(input_channels,hidden_channels, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU(),
            nn.Conv2d(hidden_channels,2*hidden_channels, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.ReLU(),
            nn.Conv2d(2*hidden_channels,hidden_channels, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU(),
        )
        self.downsample = None
        if input_channels != hidden_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(input_channels, hidden_channels, kernel_size=1, stride=1, bias=False),
                nn.BatchNorm2d(hidden_channels)
            )
            #this way we make sure that F(x) + x is doable as x and F(x)are the same size

    def forward(self,x):
        output = self.convs(x)
        if self.downsample is not None:
            x = self.downsample(x)
        return output + x # residual connection

class RockPaperScissorsResNetwork(nn.Module):
    def __init__(self, input_channels, hidden_channels, output_size, images_size, dropout_prob):
        super().__init__()     

        self.net = nn.Sequential(
            ResBlock(input_channels, hidden_channels),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            ResBlock(hidden_channels, hidden_channels),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.ReLU(),
            ResBlock(hidden_channels, hidden_channels),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(hidden_channels * images_size[0]//8 * images_size[1]//8, output_size)
        )
    
    def forward(self, X):
        logits = self.net(X)
        return logits

In [47]:
input_channels = 3
hidden_channels = 128
output_size = 3
images_size = (16,16)
dropout_prob = 0.3

model_CNN = RockPaperScissorsCNN(input_channels, hidden_channels, output_size, images_size, dropout_prob).to(device)
model_CNN.load_state_dict(torch.load("model_CNN.pth", weights_only=True))
criterion_CNN = nn.CrossEntropyLoss() # We use CrossEntropyLoss as we are solving a classification problem
optimizer_CNN = torch.optim.AdamW(model_CNN.parameters(), lr=0.0001, weight_decay=0.0001) 

model_NET = RockPaperScissorsResNetwork(input_channels, hidden_channels, output_size, images_size, dropout_prob).to(device)
model_NET.load_state_dict(torch.load("model_NET.pth", weights_only=True))

criterion_NET = nn.CrossEntropyLoss() # We use CrossEntropyLoss as we are solving a classification problem
optimizer_NET = torch.optim.AdamW(model_NET.parameters(), lr=0.001, weight_decay=0.0001) 

In [27]:
def training_loop(dataloader,model,criterion, optimizer):
    size = len(dataloader.dataset)
    total_samples_processed_in_epoch = 0 #Initialize a variable to track the total samples processed in this epoch
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred=model(X)
        loss = criterion(pred, y)

        loss.backward() # Backpropagation
        optimizer.step() #update of weights and biases  
        optimizer.zero_grad() #gradient reset 

        #Accumulate the number of samples processed in the current batch
        total_samples_processed_in_epoch += len(X) 


        if batch%10==0:
            loss_val = loss.item()
            print(f"loss: {loss_val:>7f}  [{total_samples_processed_in_epoch:>5d}/{size:>5d}]")

def evaluate_loop(dataloader, model, criterion):
    size = len(dataloader.dataset)
    elements_per_batch = len(dataloader)
    model.eval()
    sum_loss_per_batch, correct = 0, 0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)

            pred = model(X)
            sum_loss_per_batch+=criterion(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    sum_loss_per_batch/=elements_per_batch
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {sum_loss_per_batch:>8f} \n")
    
            
            


**Let's see the results !**

1) CNN model
2) ResNet model (Scroll Down to see the results)

**CNN model's results**

In [44]:
epochs = 50
scheduler_CNN = lr_scheduler.CosineAnnealingLR(optimizer_CNN, T_max=epochs, eta_min=1e-5)
for iteration in range(epochs):
    print(f"Epoch {iteration+1}\n-------------------------------")
    training_loop(training_dataloader,model_CNN,criterion_CNN,optimizer_CNN)
    evaluate_loop(evaluate_dataloader,model_CNN,criterion_CNN)
    scheduler_CNN.step()
    #display the current learning rate 
    current_lr = optimizer_CNN.param_groups[0]['lr']
    print(f"Current Learning Rate: {current_lr:.6f}")
print("Done!")

Epoch 1
-------------------------------
loss: 1.108592  [   64/ 4610]
loss: 1.105286  [  704/ 4610]
loss: 1.202035  [ 1344/ 4610]
loss: 1.003798  [ 1984/ 4610]
loss: 1.049031  [ 2624/ 4610]
loss: 1.044107  [ 3264/ 4610]
loss: 1.043493  [ 3904/ 4610]
loss: 1.089199  [ 4544/ 4610]
Test Error: 
 Accuracy: 41.7%, Avg loss: 1.095287 

Current Learning Rate: 0.000100
Epoch 2
-------------------------------
loss: 1.156426  [   64/ 4610]
loss: 1.095022  [  704/ 4610]
loss: 1.083768  [ 1344/ 4610]
loss: 1.037690  [ 1984/ 4610]
loss: 1.004380  [ 2624/ 4610]
loss: 1.009202  [ 3264/ 4610]
loss: 1.061280  [ 3904/ 4610]
loss: 1.047000  [ 4544/ 4610]
Test Error: 
 Accuracy: 47.1%, Avg loss: 1.113992 

Current Learning Rate: 0.000100
Epoch 3
-------------------------------
loss: 1.081722  [   64/ 4610]
loss: 1.032029  [  704/ 4610]
loss: 0.969922  [ 1344/ 4610]
loss: 1.033481  [ 1984/ 4610]
loss: 1.013139  [ 2624/ 4610]
loss: 1.095193  [ 3264/ 4610]
loss: 1.054376  [ 3904/ 4610]
loss: 0.981264  [ 4544

In [21]:
torch.save(model_CNN.state_dict(), "model_CNN.pth") #We save the model
print("Saved PyTorch Model State to model_CNN.pth")

Saved PyTorch Model State to model_CNN.pth


**ResNet model's results**

In [41]:
epochs = 50
scheduler_NET = lr_scheduler.CosineAnnealingLR(optimizer_NET, T_max=epochs, eta_min=1e-5)
for iteration in range(epochs):
    print(f"Epoch {iteration+1}\n-------------------------------")
    training_loop(training_dataloader,model_NET,criterion_NET,optimizer_NET)
    evaluate_loop(evaluate_dataloader,model_NET,criterion_NET)
    scheduler_NET.step()
    #display the current learning rate 
    current_lr = optimizer_NET.param_groups[0]['lr']
    print(f"Current Learning Rate: {current_lr:.6f}")
print("Done!")

Epoch 1
-------------------------------
loss: 3.006123  [   64/ 4610]
loss: 1.291084  [  704/ 4610]
loss: 1.333328  [ 1344/ 4610]
loss: 1.114921  [ 1984/ 4610]
loss: 1.014254  [ 2624/ 4610]
loss: 1.178169  [ 3264/ 4610]
loss: 1.268657  [ 3904/ 4610]
loss: 1.154929  [ 4544/ 4610]
Test Error: 
 Accuracy: 35.8%, Avg loss: 1.108067 

Current Learning Rate: 0.000999
Epoch 2
-------------------------------
loss: 1.177460  [   64/ 4610]
loss: 1.263732  [  704/ 4610]
loss: 1.064733  [ 1344/ 4610]
loss: 1.135854  [ 1984/ 4610]
loss: 1.065321  [ 2624/ 4610]
loss: 1.179522  [ 3264/ 4610]
loss: 1.073957  [ 3904/ 4610]
loss: 1.292013  [ 4544/ 4610]
Test Error: 
 Accuracy: 39.7%, Avg loss: 1.168161 

Current Learning Rate: 0.000996
Epoch 3
-------------------------------
loss: 1.120754  [   64/ 4610]
loss: 1.090776  [  704/ 4610]
loss: 1.082383  [ 1344/ 4610]
loss: 1.269287  [ 1984/ 4610]
loss: 1.079007  [ 2624/ 4610]
loss: 1.064023  [ 3264/ 4610]
loss: 1.086359  [ 3904/ 4610]
loss: 1.158544  [ 4544

In [107]:
value_to_class = {0 : 'Rock', 1 :'Paper', 2 : 'Scissors'}
n = random.randint(0, 203)

print(n)
X,y = evaluate_dataset[n]
X = X.to(device).unsqueeze(0)
model_CNN.to(device)
model_CNN.eval()
model_NET.to(device)
model_NET.eval()
with torch.no_grad():
    pred_NET = model_NET(X)
    pred_CNN = model_CNN(X)
    print(f'predicted class by ResNet : {value_to_class[pred_NET.argmax(1)[0].item()]}')
    print(f'predicted class by CNN : {value_to_class[pred_CNN.argmax(1)[0].item()]}')
    print(f'ground_truth class : {value_to_class[y]}')

163
predicted class by ResNet : Paper
predicted class by CNN : Paper
ground_truth class : Paper


In [20]:
torch.save(model_NET.state_dict(), "model_NET.pth") #We save the model
print("Saved PyTorch Model State to model_NET.pth")

Saved PyTorch Model State to model_NET.pth
